# Imports

In [ ]:
# import
import numpy as np
import matplotlib.pyplot as plt
import random

# Methods
* **generate_course**: Given a grid size, number of possible actions, and number of sand pits, generate a golf course centered around a randomly placed goal hole.

* **generate_test_course**: Similar to generate_course, except with a standardized location and size for testing purposes.

* **generate_start**: Given a Map, get a random state that is valid which will be used as a starting state.

* **in_ellipse / in_rectangle**: Check whether certain points are within our desired shape.

* **print_course**: Print out the state-reward map of the golf course.

* **print_V**: Print out the value map of the golf course.

* **step**: Takes a state and action vector (magnitude and direction) and returns the new state and reward.
      *it is important to note that if the "ball" runs into a sand pit or out of bounds, the "ball" will return to the last valid state. similarly, if the ball rolls over the goal hole, it will stop and terminate there instead of moving past it.*

In [ ]:
# methods to create shapes for the golf course
def in_ellipse(x: int, y:int, a:int, b:int, rw:int, rh:int):
    return abs((x-a)**2 / rw**2 + (y-b)**2 / rh**2) <= 1
def in_rectangle(x: int, y:int, a:int, b:int, rw:int, rh:int):
    return x >= a - rw and x <= a + rw and y >= b - rh and y <= b + rh

# method that creates the golf course
#   returns
#       - list of possible states
#       - dict of state-action values
#       - dict of state rewards
#       - terminal states
def generate_course(width:int, height:int, n_actions:int, n_pits:int):
    # declare data structures
    S_out = []
    S_tout = []
    Q_out = dict()
    Map_out = dict()
    
    # initialize states, Q and R
    for x in range(width):
        for y in range(height):
            S_out.append( (y,x) )
            Q_out[(y,x)] = [0]*n_actions    # initialize all values for all actions as 0 (putt first four, driver second four)
            Map_out[(y,x)] = 'O'               # large negative value
    
    # randomly select hole for the goal and add it to terminal states
    hole = random.choice(S_out)
    S_tout.append(hole)

    # create various shapes on course for different point values
    random_shape = [random.choice([in_ellipse, in_rectangle]) for num in range(3)]
    for x in range(width):
        for y in range(height):
            if random_shape[0](y,x,hole[0],hole[1], 3,3):
                Map_out[(y,x)] = 'G'
            elif random_shape[1](y,x,hole[0],hole[1], 8,4):
                Map_out[(y,x)] = 'R'
            elif random_shape[2](y,x,hole[0],hole[1], 8,8):
                Map_out[(y,x)] = 'G'

    # set the hole coordinate to 'H'
    Map_out[hole] = 'H'
    
    # create sand pits
    cnt_pits = 0
    while cnt_pits < n_pits:
        pit = random.choice(S_out)
        # check that the pit is not on the hole
        if pit != hole: 
            Map_out[pit] = 'P'
            cnt_pits += 1
    
    return S_out, S_tout, Q_out, Map_out

# method that creates the same course every time
def generate_test_course(width:int, height:int, n_actions:int, n_pits:int):
    # declare data structures
    S_out = []
    S_tout = []
    Q_out = dict()
    Map_out = dict()
    
    # initialize states, Q and Map
    for x in range(width):
        for y in range(height):
            S_out.append( (y,x) )
            Q_out[(y,x)] = [0]*n_actions    # initialize all values for all actions as 0 (putt first four, driver second four)
            Map_out[(y,x)] = 'O'                 # large negative value
    
    # randomly select hole for the goal and add it to terminal states
    hole = (int(height / 2), int(width * 2/3))
    S_tout.append(hole)

    # create various shapes on course for different point values
    for x in range(width):
        for y in range(height):
            if in_ellipse(y,x,hole[0],hole[1], 3,3):
                Map_out[(y,x)] = 'G'
            elif in_ellipse(y,x,hole[0],hole[1], 7,4):
                Map_out[(y,x)] = 'R'
            elif in_rectangle(y,x,hole[0],hole[1], 8,7):
                Map_out[(y,x)] = 'G'

    # set the hole coordinates
    Map_out[hole] = 'H'
    
    # create sand pits
    pits = [ (int(height/3), int(width/2)), (int(height * 3/4), int(width/2 + 3)), (int(height/2 - 2), int(width/2 + 2))]
    for pit in pits:
        Map_out[pit] = 'P'
    
    return S_out, S_tout, Q_out, Map_out

# method to find start that is in valid terrain
def generate_start(Map:dict):
    start = random.choice(list(Map.keys()))
    while Map[start] == 'O' or Map[start] == 'P' or Map[start] == 'H': 
        start = random.choice(list(Map.keys()))
    return start

## Print methods
* print_course(): state-reward map
* print_V(): value map

In [ ]:
# method that prints a formatted version of the golf course with rewards
def print_course(Map:dict):
    for x in range(WIDTH):
        for y in range(HEIGHT):
            color = ""
            if Map[(y,x)] == 'G':
                color = CYAN
            elif Map[(y,x)] == 'R':
                color = GREEN
            elif Map[(y,x)] == 'H':
                color = RED
            else:
                color = BLACK
            print(color + Map[(y,x)], end=" ")
        print()

# method that prints a formatted version of the estimated values for each state, taking the maximum of all possible actions
def print_V(Q:dict):
    for x in range(WIDTH):
        for y in range(HEIGHT):
            print(np.round(np.max(Q[(y,x)]),1), end=" ")
        print()


## Step method
parameters:
* start position
* strength (PUTT or DRIVER)
* direction 

returns:
* end position
* reward

In [ ]:
def step(start:tuple, strength:int, direction:tuple):  
    pos = [start[0], start[1]]
    reward = 0
    if Map[start] == 'P': # pits make the ball go significantly slower
        strength -= 2
    if Map[start] == 'R': # rough patches make the ball go slower at starting position
        strength -= 1
    
    # ball moves in direction by one tick of strength
    for t in range(strength):
        
        newPos = [pos[0] + direction[0], pos[1] + direction[1]]
        # print('\tIntermediate position:',tuple(newPos))
        # check if ball moves out of map. break, return last position and -1
        if tuple(newPos) not in S or Map[tuple(newPos)] == 'O':
            # print("\tOUT OF BOUNDS")
            break

        # check if ball moves into sand pit or rough - if so, return previous state and reward
        elif Map[tuple(newPos)] == 'P':
            # print("\tSAND PITS")
            pos = newPos
            break

        # check if ball moves over goal - if so, return end state and reward
        elif Map[tuple(newPos)] == 'H':
            # print("\tGOAL")
            pos = newPos
            reward = 0
            break

        # else ball keeps moving
        else:
            pos = newPos

    return tuple(pos), -1

# Variables and Constants

In [ ]:
# constants
WIDTH = 10                                      # vertical grid size
HEIGHT = 30                                     # horizontal grid size
NUM_SAND_PITS = 3                               # random terminal states 
EPSILON = 0.1                                  # the rate at which we explore
ALPHA = 0.1    
GAMMA = 1   

RED   = "\033[1;31m"  
CYAN  = "\033[1;36m"
GREEN = "\033[0;32m"
BLACK  = "\033[0;0m"

# testing
iterations = [1, 100, 1000, 10000, 100000]

# environment
terrain = ['O', 'R', 'G', 'H', 'P']             # different types of terrain on the course. 
                                                # 'OUT', 'ROUGH', 'GREEN', 'HOLE', '(Sand) Pits'
# actions
PUTT = 2                                        # strength 2
DRIVER = 3                                      # strength 3
strengths = [PUTT, DRIVER]
directions = [(-1,0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right
n_actions = len(directions) * 2                 # possible directions * possible powers

# data structures
#   -   these are generated by the generate_course method. here is an example of how to call it though.
S, S_t, Q, Map  = generate_course(WIDTH, HEIGHT, n_actions, NUM_SAND_PITS)

# Policy
Epsilon-Greedy

In [ ]:
def epsilon_greedy(Q, state, epsilon):
    be_greedy = np.random.random() < 1 - epsilon
    if be_greedy: 
        action = np.argmax(Q[state])
    else:                            
        action = np.random.randint(0, n_actions)
    
    s = int(action / len(directions))  # 0 is PUTT, 1 is DRIVER
    d = action % len(directions)     
    return s, d

# Q-Learning

In [ ]:
for it in iterations:
    S, S_t, Q, Map  = generate_test_course(WIDTH, HEIGHT, n_actions, NUM_SAND_PITS) 
    # Loop for each episode
    for i in range(it):
        state = generate_start(Map)
        # Loop for each step of episode
        while state not in S_t:
            s, d = epsilon_greedy(Q, state, EPSILON)
            newState, reward = step(state, strengths[s], directions[d])
            Q[state][s * len(directions) + d] = Q[state][s * len(directions) + d] + ALPHA * (reward + GAMMA * max(Q[newState]) - Q[state][s * len(directions) + d])
            state = newState 
    print("iteration : ", it)
    print_V(Q)

# Dyna-Q

In [ ]:
S, S_t, Q, Map = generate_test_course(WIDTH, HEIGHT, n_actions, NUM_SAND_PITS) 
Model = dict()
for s in S:
    for a in range(len(strengths)):
        for d in range(len(directions)):
            Model[s,a,d] = None

# Environment Testing

First, a demonstration of the state-reward map.

In [ ]:
print_course(Map)

We will do several demonstrations where we pick an random state *S_i* and then randomly select a strength *s* and direction *d* for our step method.

**Expected behavior**: Given a current position, the ball will 'roll' in direction *d* by *s* amount. 
There are three special cases that we should observe as well. Like in golf, the ball shall stop when it goes out of bounds (out of the array bounds), into the sand pits (denoted by reward = -99), or into the goal hole (denoted by reward = 0). 
* Out of bounds: the state that is returned will be the most recent available state, with a reward of -99.
* Sand pits: the state that is returned will be the most recent available state, with a reward of -99. 
* Goal hole: the state that is returned is the goal state, with a reward of 0. This is the terminal state and will end the episode. 